# MLOps

MLOps viene de DevOps y son las prácticas y herramientas que se utilizan para mejorar la colaboración y la comunicación entre los equipos de desarrollo de software y los equipos de operaciones.

En el caso de MLOps, se refiere a las prácticas y herramientas que se utilizan para mejorar la colaboración y la comunicación entre los equipos de desarrollo de modelos de machine learning y los equipos de operaciones

## Estapas de MLOps

Estas son las etapas de MLOps:

![Etapas MLOps](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlops_operations.webp)

Vamos a explicarlas:

 * Diseño: En esta etapa se define el problema que se va a resolver y se recopilan los datos necesarios para resolverlo.
 * Desarrollo del modelo: En esta etapa se entrena el modelo y se evalúa su rendimiento.
 * Operaciones: En esta etapa se despliega el modelo en producción, se crean los pipelines de CI/CD y se monitoriza el rendimiento del modelo.

Como se puede ver no son etapas cerradas, sino que nos podemos mover de una a otra en cualquier momento como se puede ver en la siguiente imagen:

![mlops operations](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlops_operations2.webp)